In [1]:
from langchain.llms import GooglePalm

In [2]:
### Generate your own google api key at: https://makersuite.google.com/app/apikey
api = 
llm = GooglePalm(google_api_key=api)

In [31]:
# Test Language model
poem = llm('Writea 4 line poem about potatoes')
print(poem)

**Potatoes**

Round and white,
A starchy delight,
Roast 'em, mash 'em,
Stick 'em in a stew.


In [4]:
from langchain.document_loaders.csv_loader import CSVLoader

In [5]:
# Load Data base
loader = CSVLoader(file_path='codebasics_faqs.csv', source_column='prompt', encoding='latin1')
data = loader.load()

In [10]:
from langchain.embeddings import HuggingFaceInstructEmbeddings
from langchain.vectorstores import FAISS

In [12]:
# Generate emebedding and vector data base (5min runtime)
instructor_embeddings = HuggingFaceInstructEmbeddings(query_instruction='Retrieve the query for retrieval: ')
vector_db = FAISS.from_documents(documents=data, embedding=instructor_embeddings)
retriever = vector_db.as_retriever(score_threshold= 0.7)

load INSTRUCTOR_Transformer
max_seq_length  512


In [35]:
# Test the Chatbot
rdocs = retriever.get_relevant_documents('how about job placements?')
rdocs

[Document(page_content='prompt: Do you provide any job assistance?\nresponse: Yes, We help you with resume and interview preparation along with that we help you in building online credibility, and based on requirements we refer candidates to potential recruiters.', metadata={'source': 'Do you provide any job assistance?', 'row': 11}),
 Document(page_content='prompt: Will this course guarantee me a job?\nresponse: We created a much lighter version of this course on YouTube available for free (click this link) and many people gave us feedback that they were able to fetch jobs (see testimonials). Now this paid course is at least 5x better than the YouTube course which gives us ample confidence that you will be able to get a job. However, we want to be honest and do not want to make any impractical promises! Our guarantee is to prepare you for the job market by teaching the most relevant skills, knowledge & timeless principles good enough to fetch the job.', metadata={'source': 'Will this 

In [36]:
from langchain.prompts import PromptTemplate   
from langchain.chains import RetrievalQA 

# If no match in database, the chatbot should answer 'I don't know'
prompt_template = """Given the following context and a question, generate an answer based on this context only.
In the answer try to provide as much text as possible from "response" section in the source document context without making much changes.
If the answer is not found in the context, kindly state "I don't know." Don't try to make up an answer.

CONTEXT: {context}

QUESTION: {question}"""


PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)
chain_type_kwargs = {"prompt": PROMPT}

from langchain.chains import RetrievalQA

# Create function to query questions and retriev results
chain = RetrievalQA.from_chain_type(llm=llm,
                            chain_type="stuff",
                            retriever=retriever,
                            input_key="query",
                            return_source_documents=True,
                            chain_type_kwargs=chain_type_kwargs)

In [37]:
# Test on multiple questions
chain('Do you guys provide internship and also do you offer EMI payments?')

{'query': 'Do you guys provide internship and also do you offer EMI payments?',
 'result': 'Yes, we provide virtual internship and we do not offer EMI payments.',
 'source_documents': [Document(page_content='prompt: Do you provide any virtual internship?\nresponse: Yes', metadata={'source': 'Do you provide any virtual internship?', 'row': 14}),
  Document(page_content='prompt: Do we have an EMI option?\nresponse: No', metadata={'source': 'Do we have an EMI option?', 'row': 13}),
  Document(page_content='prompt: Do you provide any job assistance?\nresponse: Yes, We help you with resume and interview preparation along with that we help you in building online credibility, and based on requirements we refer candidates to potential recruiters.', metadata={'source': 'Do you provide any job assistance?', 'row': 11}),
  Document(page_content='prompt: Will this bootcamp guarantee me a job?\nresponse: The courses included in this bootcamp are done by 9000+ learners and many of them have secured 

In [40]:
# Test response if question is not in database
chain('Do you have javascript courses?')

{'query': 'Do you have javascript courses?',
 'result': "I don't know.",
 'source_documents': [Document(page_content='prompt: I have never done programming and belong to a non-technical background. Can I take this course?\nresponse: Yes, this is the perfect course for anyone who has never done coding and wants to build a career in the IT/Data Analytics industry or just wants to perform better in their current job or business using data.', metadata={'source': 'I have never done programming and belong to a non-technical background. Can I take this course?', 'row': 24}),
  Document(page_content='prompt: Is there any prerequisite for taking this bootcamp ?\nresponse: Our bootcamp is specifically designed for beginners with no prior experience in this field. The only prerequisite is that you need to have a functional laptop with at least 4GB ram, an internet connection, and a thrill to learn data analysis.', metadata={'source': 'Is there any prerequisite for taking this bootcamp ?', 'row': 